In [2]:
import jax
import jax.numpy as jnp
from functools import partial


In [17]:
def f(x,val: int):
    print("type of x",type(x))
    print("type of val",type(val))
    return x.sum() + val




In [18]:
jitted_f = jax.jit(f,static_argnames=["val"])

In [19]:
out = jitted_f(jnp.array([1, 2, 3]), 10)

type of x <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>
type of val <class 'int'>


In [22]:
out.item()


16

In [3]:
rng = jax.random.PRNGKey(0)
rng, rng_split = jax.random.split(rng)
rng_split, rng_split_2 = jax.random.split(rng_split)
rng_split_2, rng_split_3 = jax.random.split(rng_split_2)
rng_split_3, rng_split_4 = jax.random.split(rng_split_3)
rng_split_4, rng_split_5 = jax.random.split(rng_split_4)
rng_split_5, rng_split_6 = jax.random.split(rng_split_5)

In [10]:
if isinstance(rng, jnp.ndarray):
    print("rng is a jnp.ndarray")
else:
    print("rng is not a jnp.ndarray")



rng is a jnp.ndarray


In [11]:
print(type(rng))

<class 'jaxlib.xla_extension.ArrayImpl'>
